# Итераторы

Для понимания, что делает `yield`, необходимо понимать, что такое генераторы. Генераторам же предшествуют итераторы. 

Когда вы создаёте список, вы можете считывать его элементы один за другим — это называется **итерацией**:

In [2]:
mylist = [1, 2, 3]

for i in mylist:
    print(i)

1
2
3


*Mylist* является **итерируемым объектом**. Когда вы создаёте список, используя генераторное выражение, вы создаёте также итератор:

In [3]:
mylist = [x*x for x in range(3)]

for i in mylist:
    print(i)

0
1
4


Всё, к чему можно применить конструкцию `«for… in...»`, является итерируемым объектом: списки, строки, файлы… 

Это удобно, потому что можно считывать из них значения сколько потребуется — однако все значения хранятся в памяти, а это не всегда желательно, если у вас много значений.

# Генераторы

**Генераторы** - это тоже итерируемые объекты, но прочитать их можно лишь один раз. Это связано с тем, что они не хранят значения в памяти, а генерируют их на лету:

In [4]:
mygenerator = (x*x for x in range(3))

for i in mygenerator:
    print(i)

0
1
4


Всё то же самое, разве что используются круглые скобки вместо квадратных. 

**НО**: нельзя применить конструкцию `for i in mygenerator` второй раз, так как генератор может быть использован только единожды: 
- он вычисляет 0; 
- потом забывает про него и вычисляет 1; 
- завершает вычислением 4 — одно за другим.

# Yield

*Yield* — это ключевое слово, которое используется примерно как return — отличие в том, что функция вернёт генератор.

In [13]:
def create_generator():
    for i in range(3):
        yield i*i

mygenerator = create_generator()
print(mygenerator) # mygenerator - объект генератора!
for i in mygenerator:
    print(i)

<generator object create_generator at 0x000001D02A625F10>
0
1
4


In [14]:
for i in mygenerator:
    print(i)

В данном случае пример бесполезный, но это удобно, если вы знаете, что функция вернёт большой набор значений, который надо будет прочитать только один раз.

Чтобы освоить `yield`, вы должны понимать, что когда вы вызываете функцию, код внутри тела функции не исполняется. Функция только возвращает объект-генератор — немного мудрёно :-)


Теперь трудная часть:

В первый запуск вашей функции, она будет исполняться от начала до того момента, когда она наткнётся на `yield` — тогда она вернёт первое значение из цикла. На каждый следующий вызов будет происходить ещё одна итерация написанного вами цикла, возвращаться будет следующее значение — и так пока значения не кончатся.

Генератор считается пустым, как только при исполнении кода функции не встречается `yield`. Это может случиться из-за конца цикла, или же если не выполняется какое-то из условий `if/else`.

In [15]:
result, candidates = list(), [self]
candidates

NameError: name 'self' is not defined

# Объяснение кода

Генератор:

In [16]:
# Создаём метод узла, который будет возвращать генератор
def _get_child_candidates(self, distance, min_dist, max_dist):

  # Этот код будет вызываться при каждом обращении к объекту-генератору:

  # Если у узла есть потомок слева
  # И с расстоянием всё в порядке, возвращаем этого потомка
    if self._leftchild and distance - max_dist < self._median:
                yield self._leftchild

  # Если у узла есть потомок справа
  # И с расстоянием всё в порядке, возвращаем этого потомка
    if self._rightchild and distance + max_dist >= self._median:
                yield self._rightchild

  # Если исполнение дошло до этого места, генератор считается пустым

In [ ]:
# Создаём пустой список и список со ссылкой на текущий объект
result, candidates = list(), [self]

# Входим в цикл по кандидатам (в начале там только один элемент)
while candidates:

    # Вытягиваем последнего кандидата и удаляем его из списка
    node = candidates.pop()

    # Вычисляем расстояние между объектом и кандидатом
    distance = node._get_dist(obj)

    # Если с расстоянием всё в порядке, добавляем в результат
    if distance <= max_dist and distance >= min_dist:
        result.extend(node._values)

    # Добавляем потомков кандидата в список кандидатов,
    # чтобы цикл продолжал исполняться до тех пор,
    # пока не обойдёт всех потомков потомков <...> кандидата
    candidates.extend(node._get_child_candidates(distance, min_dist, max_dist))

return result

Этот код содержит несколько меньших частей:

- Цикл итерируется по списку, но списко расширяется во время итерации :-) Это лаконичный способ обойти все сгрупиррованные данные, зоть это и немного опасно, так как может обернуться бесконечным циклом. В таком случае `candidates.extend(node._get_child_candidates(distance, min_dist, max_dist))` исчерпает все значения генератора, но при этом продолжит создавать новые объекты-генераторы, которые будут давать значения, отличные от предыдущих (поскольку применяются к к другим узлам).

- Метод `extend()` это метод объекта списка, который ожидает на вход что-нибудь итерируемое и добавляет его значения к списку.

Обычно мы передаём ему список:

In [17]:
a = [1, 2]
b = [3, 4]

a.extend(b)
print(a)

[1, 2, 3, 4]


Но в нашем коде он принимает генератор, что хорошо по следующим причинам:

- Нет необходимости читать значения дважды.
- Может случиться так, что потомков много и хранить их всех в памяти не хочется.

И это работает, потому что Python всё равно, является аргумент этого метода списком или нет. Python ожидает итерируемый объект, так что это сработает со строками, списками, кортежами и генераторами! Это называется утиной типизацией и является одной из причин, почему Python так крут. Но это другая история для другого вопроса…

# Контроль за исчерпанием генератора

In [24]:
# Создаем банк, строящий торговые автоматы
# ATM - Automatic Teller Machine
class Bank:
    crisis = False
    def create_atm(self):
        while not self.crisis:
            yield '$100'

In [25]:
hsbc = Bank()
# Когда все хорошо, можно получить деньги с ATM
corner_street_atm = hsbc.create_atm()
print(corner_street_atm.__next__())

$100


In [26]:
print(corner_street_atm.__next__())

$100


In [27]:
print([corner_street_atm.__next__() for cash in range(5)])

['$100', '$100', '$100', '$100', '$100']


In [28]:
# Пришел кризис, денег нет
hsbc.crisis = True
print(corner_street_atm.__next__())

StopIteration: 

In [29]:
# Что верно даже для новых автоматов:
wall_street_atm = hsbc.create_atm()
print(wall_street_atm.__next__())

StopIteration: 

In [30]:
# Проблема в том, что когда кризис прошел, автоматы все еще пустые:
hsbc.crisis = False
print(corner_street_atm.__next__())

StopIteration: 

In [32]:
# Но если построить еще один, то все ок
brand_new_atm = hsbc.create_atm()
print([brand_new_atm.__next__() for cash in range(4)])

['$100', '$100', '$100', '$100']


Это может оказаться полезным для разных целей вроде управления доступом к какому-нибудь ресурсу.